In [1]:
import kfp, os
import kfp.dsl as dsl 
import kfp.compiler as compiler
import kubernetes.client.models as k8s
from kfp.aws import use_aws_secret
import namesgenerator
from utils import *

In [2]:
NAMESPACE = os.environ.get("NAMESPACE") 

kubeflow_address = f"http://{NAMESPACE}.kubeflow.odsc.k8s.hydrosphere.io"
hydrosphere_address = f"http://{NAMESPACE}.serving.odsc.k8s.hydrosphere.io"

In [ ]:
print(kubeflow_address)
print(hydrosphere_address)

# 01 Download Data

### Pipeline

At this stage we will obtain all training data for our pipeline.

In [4]:
username = "hydrosphere"
version = "v2"

In [5]:
def download_op(**kwargs):
    return dsl.ContainerOp(
        name="download",
        image=f"{username}/mnist-pipeline-download:{version}",
        file_outputs={"data_path": "/data_path.txt"},
        arguments=["--hydrosphere-address", kwargs["hydrosphere_address"]]
    ).apply(use_aws_secret())

Create pipeline definition.

In [6]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(hydrosphere_address):
    
    # 01 Download Data
    download = download_op(
        hydrosphere_address=hydrosphere_address)

Compile pipeline.

In [7]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [10]:
# Create Pipelines client
client = kfp.Client(kubeflow_address)

In [11]:
# Define an experiment name
experiment_name='MNIST Showreal'

In [12]:
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id

In [13]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name modest_sammet


# 02 Train Model & Autoencoder

### Pipeline

At this stage we will create a model and an autoencoder & train them on the downloaded data.

In [14]:
def train_model_op(**kwargs):
    return dsl.ContainerOp(
        name="train_model",
        image=f"{username}/mnist-pipeline-train-model:{version}",
        file_outputs={
            "accuracy": "/accuracy.txt",
            "model_path": "/model_path.txt",
            "classes": "/classes.txt",
        },
        arguments=[
            "--data-path", kwargs["data_path"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"],
            "--hydrosphere-address", kwargs["hydrosphere_address"]
        ],
    ).apply(use_aws_secret())

In [15]:
def train_autoencoder_op(**kwargs):
    return dsl.ContainerOp(
        name="train_autoencoder",
        image=f"{username}/mnist-pipeline-train-autoencoder:{version}",
        file_outputs={
            "model_path": "/model_path.txt",
            "loss": "/loss.txt",
            "classes": "/classes.txt",
        },
        arguments=[
            "--data-path", kwargs["data_path"], 
            "--steps", kwargs["steps"], 
            "--learning-rate", kwargs["learning_rate"],
            "--batch-size", kwargs["batch_size"],
            "--hydrosphere-address", kwargs["hydrosphere_address"]
        ]
    ).apply(use_aws_secret())

Update pipeline definition.

In [16]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    model_learning_rate,
    model_epochs,
    model_batch_size,
    autoencoder_learning_rate,
    autoencoder_steps,
    autoencoder_batch_size,
):
    # 01 Download Data
    download = download_op(
        hydrosphere_address=hydrosphere_address)
    
    # 02 Train Model & Autoencoder
    train_model = train_model_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    train_model.set_memory_request('1G')
    train_model.set_cpu_request('1')
    
    train_autoencoder = train_autoencoder_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"],
        learning_rate=autoencoder_learning_rate,
        batch_size=autoencoder_batch_size,
        steps=autoencoder_steps)
    train_autoencoder.set_memory_request('1G')
    train_autoencoder.set_cpu_request('1')

Compile pipeline.

In [17]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [19]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere_address": hydrosphere_address,
        "model_learning_rate": "0.01",
        "model_epochs": "10",
        "model_batch_size": "256",
        "autoencoder_learning_rate": "0.01",
        "autoencoder_steps": "3500",
        "autoencoder_batch_size": "256",
    }
)

Starting a new run with the name vibrant_goldwasser


# 03 Release & Deploy Autoencoder

### Pipeline

At this stage we will release and deploy the trained autoencoder to the cluster.

In [20]:
def release_autoencoder_op(**kwargs):
    return dsl.ContainerOp(
        name="release_autoencoder",
        image=f"{username}/mnist-pipeline-release-autoencoder:{version}",
        file_outputs={
            "model_version": "/model_version.txt",
            "model_link": "/model_link.txt"
        },
        arguments=[
            "--data-path", kwargs["data_path"],
            "--model-name", kwargs["model_name"],
            "--models-path", kwargs["models_path"],
            "--classes", kwargs["classes"],
            "--loss", kwargs["loss"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--learning-rate", kwargs["learning_rate"],
            "--batch-size", kwargs["batch_size"],
            "--steps", kwargs["steps"]
        ],
    ).apply(use_aws_secret())

In [21]:
def deploy_op(**kwargs):
    return dsl.ContainerOp(
        name=kwargs["name"],
        image=f"{username}/mnist-pipeline-deploy:{version}",
        file_outputs={
            "application_name": "/application_name.txt",
            "application_link": "/application_link.txt"
        },
        arguments=[
            "--model-name", kwargs["model_name"],
            "--model-version", kwargs["model_version"],
            "--application-name-postfix", kwargs["postfix"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [22]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    model_learning_rate,
    model_epochs,
    model_batch_size,
    autoencoder_learning_rate,
    autoencoder_steps,
    autoencoder_batch_size,
    model_name,
    model_autoencoder_name,
):
    # 01 Download Data
    download = download_op(
        hydrosphere_address=hydrosphere_address)
    
    # 02 Train Model & Autoencoder
    train_model = train_model_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    train_model.set_memory_request('1G')
    train_model.set_cpu_request('1')
    
    train_autoencoder = train_autoencoder_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"],
        learning_rate=autoencoder_learning_rate,
        batch_size=autoencoder_batch_size,
        steps=autoencoder_steps)
    train_autoencoder.set_memory_request('1G')
    train_autoencoder.set_cpu_request('1')
    
    # 03 Release & Deploy Autoencoder
    release_autoencoder = release_autoencoder_op(
        data_path=download.outputs["data_path"],
        model_name=model_autoencoder_name,
        models_path=train_autoencoder.outputs["model_path"],
        classes=train_autoencoder.outputs["classes"],
        loss=train_autoencoder.outputs["loss"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=autoencoder_learning_rate,
        steps=autoencoder_steps,
        batch_size=autoencoder_batch_size)
    
    deploy_autoencoder_to_prod = deploy_op(
        name="deploy_autoencoder_to_prod",
        model_name=model_autoencoder_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_autoencoder.outputs["model_version"])

Compile pipeline.

In [23]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [25]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "model_learning_rate": "0.01",
        "model_epochs": "10",
        "model_batch_size": "256",
        "autoencoder_learning_rate": "0.01",
        "autoencoder_steps": "3500",
        "autoencoder_batch_size": "256",
        "model_name": "mnist",
        "model_autoencoder_name": "mnist_autoencoder",
    }
)

Starting a new run with the name pedantic_bell


# 04 Release & Deploy Model to Stage

### Pipeline

At this stage we release the model and deploy it to the stage application for integration tests.

In [32]:
def release_model_op(**kwargs): 
    return dsl.ContainerOp(
        name="release_model",
        image=f"{username}/mnist-pipeline-release-model:{version}", 
        file_outputs={
            "model_version": "/model_version.txt",
            "model_link": "/model_link.txt"
        },
        arguments=[
            "--data-path", kwargs["data_path"],
            "--model-name", kwargs["model_name"],
            "--models-path", kwargs["models_path"],
            "--autoencoder-app", kwargs["application_name"],
            "--classes", kwargs["classes"],
            "--accuracy", kwargs["accuracy"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"],
        ]
    ).apply(use_aws_secret())

Update pipeline definition.

In [33]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    model_learning_rate,
    model_epochs,
    model_batch_size,
    autoencoder_learning_rate,
    autoencoder_steps,
    autoencoder_batch_size,
    model_name,
    model_autoencoder_name,
):
    # 01 Download Data
    download = download_op(
        hydrosphere_address=hydrosphere_address)
    
    # 02 Train Model & Autoencoder
    train_model = train_model_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    train_model.set_memory_request('1G')
    train_model.set_cpu_request('1')
    
    train_autoencoder = train_autoencoder_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"],
        learning_rate=autoencoder_learning_rate,
        batch_size=autoencoder_batch_size,
        steps=autoencoder_steps)
    train_autoencoder.set_memory_request('1G')
    train_autoencoder.set_cpu_request('1')
    
    # 03 Release & Deploy Autoencoder
    release_autoencoder = release_autoencoder_op(
        data_path=download.outputs["data_path"],
        model_name=model_autoencoder_name,
        models_path=train_autoencoder.outputs["model_path"],
        classes=train_autoencoder.outputs["classes"],
        loss=train_autoencoder.outputs["loss"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=autoencoder_learning_rate,
        steps=autoencoder_steps,
        batch_size=autoencoder_batch_size)
    
    deploy_autoencoder_to_prod = deploy_op(
        name="deploy_autoencoder_to_prod",
        model_name=model_autoencoder_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_autoencoder.outputs["model_version"])
    
    # 04 Release & Deploy Model to Stage
    release_model = release_model_op(
        data_path=download.outputs["data_path"],
        model_name=model_name,
        models_path=train_model.outputs["model_path"],
        application_name=deploy_autoencoder_to_prod.outputs["application_name"],
        classes=train_model.outputs["classes"],
        accuracy=train_model.outputs["accuracy"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)

    deploy_model_to_stage = deploy_op(
        name="deploy_model_to_stage",
        model_name=model_name,
        postfix="_stage",
        hydrosphere_address=hydrosphere_address,
        model_version=release_model.outputs["model_version"])

Compile pipeline.

In [34]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [36]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "model_learning_rate": "0.01",
        "model_epochs": "10",
        "model_batch_size": "256",
        "autoencoder_learning_rate": "0.01",
        "autoencoder_steps": "3500",
        "autoencoder_batch_size": "256",
        "model_name": "mnist",
        "model_autoencoder_name": "mnist_autoencoder",
    }
)

Starting a new run with the name silly_northcutt


# 05 Test Stage Application

### Pipeline

At this stage we are performing integration tests.

In [38]:
def test_op(**kwargs):
    return dsl.ContainerOp(
        name="test",
        image=f"{username}/mnist-pipeline-test:{version}", 
        arguments=[
            "--data-path", kwargs["data_path"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--acceptable-accuracy", kwargs["acceptable_accuracy"],
            "--application-name", kwargs["application_name"], 
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [39]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    model_learning_rate,
    model_epochs,
    model_batch_size,
    autoencoder_learning_rate,
    autoencoder_steps,
    autoencoder_batch_size,
    model_name,
    model_autoencoder_name,
    acceptable_accuracy
):
    # 01 Download Data
    download = download_op(
        hydrosphere_address=hydrosphere_address)
    
    # 02 Train Model & Autoencoder
    train_model = train_model_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    train_model.set_memory_request('1G')
    train_model.set_cpu_request('1')
    
    train_autoencoder = train_autoencoder_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"],
        learning_rate=autoencoder_learning_rate,
        batch_size=autoencoder_batch_size,
        steps=autoencoder_steps)
    train_autoencoder.set_memory_request('1G')
    train_autoencoder.set_cpu_request('1')
    
    # 03 Release & Deploy Autoencoder
    release_autoencoder = release_autoencoder_op(
        data_path=download.outputs["data_path"],
        model_name=model_autoencoder_name,
        models_path=train_autoencoder.outputs["model_path"],
        classes=train_autoencoder.outputs["classes"],
        loss=train_autoencoder.outputs["loss"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=autoencoder_learning_rate,
        steps=autoencoder_steps,
        batch_size=autoencoder_batch_size)
    
    deploy_autoencoder_to_prod = deploy_op(
        name="deploy_autoencoder_to_prod",
        model_name=model_autoencoder_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_autoencoder.outputs["model_version"])
    
    # 04 Release & Deploy Model to Stage
    release_model = release_model_op(
        data_path=download.outputs["data_path"],
        model_name=model_name,
        models_path=train_model.outputs["model_path"],
        application_name=deploy_autoencoder_to_prod.outputs["application_name"],
        classes=train_model.outputs["classes"],
        accuracy=train_model.outputs["accuracy"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    
    deploy_model_to_stage = deploy_op(
        name="deploy_model_to_stage",
        model_name=model_name,
        postfix="_stage",
        hydrosphere_address=hydrosphere_address,
        model_version=release_model.outputs["model_version"])
    
    # 05 Test Stage Application
    test = test_op(
        data_path=download.outputs["data_path"],
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        application_name=deploy_model_to_stage.outputs["application_name"])
    test.set_retry(3)

Compile pipeline.

In [40]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [43]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "model_learning_rate": "0.01",
        "model_epochs": "10",
        "model_batch_size": "256",
        "autoencoder_learning_rate": "0.01",
        "autoencoder_steps": "3500",
        "autoencoder_batch_size": "256",
        "model_name": "mnist",
        "model_autoencoder_name": "mnist_autoencoder",
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name sleepy_hoover


# 06 Deploy Model to Production

### Pipeline

And finally deploying the model to production application

Update pipeline definition.

In [44]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    model_learning_rate,
    model_epochs,
    model_batch_size,
    autoencoder_learning_rate,
    autoencoder_steps,
    autoencoder_batch_size,
    model_name,
    model_autoencoder_name,
    acceptable_accuracy
):
    # 01 Download Data
    download = download_op(
        hydrosphere_address=hydrosphere_address)
    
    # 02 Train Model & Autoencoder
    train_model = train_model_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    train_model.set_memory_request('1G')
    train_model.set_cpu_request('1')
    
    train_autoencoder = train_autoencoder_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"],
        learning_rate=autoencoder_learning_rate,
        batch_size=autoencoder_batch_size,
        steps=autoencoder_steps)
    train_autoencoder.set_memory_request('1G')
    train_autoencoder.set_cpu_request('1')
    
    # 03 Release & Deploy Autoencoder
    release_autoencoder = release_autoencoder_op(
        data_path=download.outputs["data_path"],
        model_name=model_autoencoder_name,
        models_path=train_autoencoder.outputs["model_path"],
        classes=train_autoencoder.outputs["classes"],
        loss=train_autoencoder.outputs["loss"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=autoencoder_learning_rate,
        steps=autoencoder_steps,
        batch_size=autoencoder_batch_size)
    
    deploy_autoencoder_to_prod = deploy_op(
        name="deploy_autoencoder_to_prod",
        model_name=model_autoencoder_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_autoencoder.outputs["model_version"])
    
    # 04 Release & Deploy Model to Stage
    release_model = release_model_op(
        data_path=download.outputs["data_path"],
        model_name=model_name,
        models_path=train_model.outputs["model_path"],
        application_name=deploy_autoencoder_to_prod.outputs["application_name"],
        classes=train_model.outputs["classes"],
        accuracy=train_model.outputs["accuracy"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    
    deploy_model_to_stage = deploy_op(
        name="deploy_model_to_stage",
        model_name=model_name,
        postfix="_stage",
        hydrosphere_address=hydrosphere_address,
        model_version=release_model.outputs["model_version"])
    
    # 05 Test Stage Application
    test = test_op(
        data_path=download.outputs["data_path"],
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        application_name=deploy_model_to_stage.outputs["application_name"])
    test.set_retry(3)
    
    # 06 Deploy Model to Production
    deploy_model_to_prod = deploy_op(
        name="deploy_model_to_prod", 
        model_name=model_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_model.outputs["model_version"])
    deploy_model_to_prod.after(test)

Compile pipeline.

In [45]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [47]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "model_learning_rate": "0.01",
        "model_epochs": "10",
        "model_batch_size": "256",
        "autoencoder_learning_rate": "0.01",
        "autoencoder_steps": "3500",
        "autoencoder_batch_size": "256",
        "model_name": "mnist",
        "model_autoencoder_name": "mnist_autoencoder",
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name angry_turing


## Production traffic

In [ ]:
simulate_production_traffic(shuffle=False)

# 01 Sample Production Data

### Pipeline

Once we have a fully functional pipeline, we would like to automate running this pipeline.

In [50]:
def sample_op(**kwargs):
    return dsl.ContainerOp(
        name="sample",
        image=f"{username}/mnist-pipeline-sample:{version}", 
        file_outputs={"data_path": "/data_path.txt"},
        arguments=[
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--application-name", kwargs["application_name"],
        ]
    ).apply(use_aws_secret())

Update pipeline definition.

In [51]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    model_learning_rate,
    model_epochs,
    model_batch_size,
    autoencoder_learning_rate,
    autoencoder_steps,
    autoencoder_batch_size,
    model_name,
    application_name,
    model_autoencoder_name,
    acceptable_accuracy
):
    # 01 Sample Data
    sample = sample_op(
        hydrosphere_address=hydrosphere_address,
        application_name=application_name)
    
    # 02 Train Model & Autoencoder
    train_model = train_model_op(
        hydrosphere_address=hydrosphere_address,
        data_path=sample.outputs["data_path"], 
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    train_model.set_memory_request('1G')
    train_model.set_cpu_request('1')
    
    train_autoencoder = train_autoencoder_op(
        hydrosphere_address=hydrosphere_address,
        data_path=sample.outputs["data_path"],
        learning_rate=autoencoder_learning_rate,
        batch_size=autoencoder_batch_size,
        steps=autoencoder_steps)
    train_autoencoder.set_memory_request('1G')
    train_autoencoder.set_cpu_request('1')
    
    # 03 Release & Deploy Autoencoder
    release_autoencoder = release_autoencoder_op(
        data_path=sample.outputs["data_path"],
        model_name=model_autoencoder_name,
        models_path=train_autoencoder.outputs["model_path"],
        classes=train_autoencoder.outputs["classes"],
        loss=train_autoencoder.outputs["loss"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=autoencoder_learning_rate,
        steps=autoencoder_steps,
        batch_size=autoencoder_batch_size)
    
    deploy_autoencoder_to_prod = deploy_op(
        name="deploy_autoencoder_to_prod",
        model_name=model_autoencoder_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_autoencoder.outputs["model_version"])
    
    # 04 Release & Deploy Model to Stage
    release_model = release_model_op(
        data_path=sample.outputs["data_path"],
        model_name=model_name,
        models_path=train_model.outputs["model_path"],
        application_name=deploy_autoencoder_to_prod.outputs["application_name"],
        classes=train_model.outputs["classes"],
        accuracy=train_model.outputs["accuracy"],
        hydrosphere_address=hydrosphere_address,
        learning_rate=model_learning_rate,
        epochs=model_epochs,
        batch_size=model_batch_size)
    
    deploy_model_to_stage = deploy_op(
        name="deploy_model_to_stage",
        model_name=model_name,
        postfix="_stage",
        hydrosphere_address=hydrosphere_address,
        model_version=release_model.outputs["model_version"])
    
    # 05 Test Stage Application
    test = test_op(
        data_path=sample.outputs["data_path"],
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        application_name=deploy_model_to_stage.outputs["application_name"])
    test.set_retry(3)
    
    # 06 Deploy Model to Production
    deploy_model_to_prod = deploy_op(
        name="deploy_model_to_prod", 
        model_name=model_name,
        postfix="_app",
        hydrosphere_address=hydrosphere_address,
        model_version=release_model.outputs["model_version"])
    deploy_model_to_prod.after(test)

Compile pipeline.

In [53]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [ ]:
%%bash
tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml

### Test Run

In [56]:
# start a run
run_name = namesgenerator.get_random_name() + "_sample"
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "model_learning_rate": "0.0005",
        "model_epochs": "100",
        "model_batch_size": "256",
        "autoencoder_learning_rate": "0.01",
        "autoencoder_steps": "3500",
        "autoencoder_batch_size": "128",
        "model_name": "mnist",
        "model_autoencoder_name": "mnist_autoencoder",
        "acceptable-accuracy": "0.80",
        "application_name": "mnist_app", 
    }
)

Starting a new run with the name sharp_bose_sample


## Production traffic

In [ ]:
simulate_production_traffic(shuffle=False)